In [ ]:
pip install natsort==8.4.0

In [ ]:
pip install einops==0.6.1

In [ ]:
#导入上传的文件，一定要运行
import sys
sys.path.append('../input/smartphone-image-denoising-dataset')
sys.path.append('../input/uformer-main')
print("导入成功")

# 上面的内容每次都要重新运行

# 切换到Input目录

In [ ]:
'''
import os

# 切换到指定路径Input路径
new_path = '../input'
os.chdir(new_path)
# 打印当前工作目录
print(os.getcwd())

'''

# 切换到Output下面的working目录

In [ ]:
'''
new_path = "../working/"
os.chdir(new_path)
# 打印当前工作目录
print(os.getcwd())
'''

# 创建内容都要在Output下面

In [ ]:
'''# 使用 os 模块创建文件夹
import os
with open('../working/output.txt', 'w') as f:
    f.write('Hello!')
'''

# 删除output中的内容

In [ ]:
'''
import shutil
import os
 
if __name__ == '__main__':
    path = '/kaggle/working/'
    if os.path.exists(path):
        shutil.rmtree(path)
        print('删除完成')
    else:
        print('原本为空')
'''

# 上面是一些基本操作的功能

# 对应Github上 
## python3 generate_patches_SIDD.py --src_dir ../SIDD_Medium_Srgb/Data --tar_dir ../datasets/denoising/sidd/train 这里的操作


In [ ]:

from glob import glob
from tqdm import tqdm
import numpy as np
import os
from natsort import natsorted
import cv2
from joblib import Parallel, delayed
import multiprocessing
import argparse



#num_patches =300, num_cores=4

src = "/kaggle/input/smartphone-image-denoising-dataset/SIDD_Small_sRGB_Only/Data"
tar = "/kaggle/working/datasets/denoising/sidd/train"
PS = 256
NUM_PATCHES = 300
NUM_CORES = 4

noisy_patchDir = os.path.join(tar, 'input')
clean_patchDir = os.path.join(tar, 'groundtruth')

os.makedirs(noisy_patchDir)
os.makedirs(clean_patchDir)
print("执行了")





In [ ]:

files = natsorted(glob(os.path.join(src, '*', '*.PNG')))


In [ ]:

noisy_files, clean_files = [], []
for file_ in files:
    filename = os.path.split(file_)[-1]
    if 'GT' in filename:
        clean_files.append(file_)
    if 'NOISY' in filename:
        noisy_files.append(file_)

def save_files(i):
    noisy_file, clean_file = noisy_files[i], clean_files[i]
    noisy_img = cv2.imread(noisy_file)
    clean_img = cv2.imread(clean_file)

    H = noisy_img.shape[0]
    W = noisy_img.shape[1]
    for j in range(NUM_PATCHES):
        rr = np.random.randint(0, H - PS)
        cc = np.random.randint(0, W - PS)
        noisy_patch = noisy_img[rr:rr + PS, cc:cc + PS, :]
        clean_patch = clean_img[rr:rr + PS, cc:cc + PS, :]

        cv2.imwrite(os.path.join(noisy_patchDir, '{}_{}.png'.format(i+1,j+1)), noisy_patch)
        cv2.imwrite(os.path.join(clean_patchDir, '{}_{}.png'.format(i+1,j+1)), clean_patch)

print("搞完了")
print(len(noisy_files))

              

In [ ]:

Parallel(n_jobs=4)(delayed(save_files)(i) for i in tqdm(range(len(noisy_files))))  



# 生成对应的denoising数据集的工作已经完成 ，下面开始Training 模块中的Denoising 
## 下面的代码对应的是 sh script/train_denoise.sh

In [ ]:
class Options():
    batch_size=1
    nepoch=10      #默认250轮
    train_workers=4
    eval_workers=4
    dataset="SIDD"
    pretrain_weights='./log/Uformer_B/models/model_best.pth'
    optimizer='adamw'
    lr_initial=0.0002
    step_lr=50
    weight_decay=0.02
    gpu='0,1'
    arch="Uformer_B"
    mode='denoising'
    dd_in=3
    save_dir='./logs/'
    save_images=False
    env='_'
    checkpoint=50
    norm_layer='nn.LayerNorm'
    embed_dim=32
    win_size=8
    token_projection='linear'
    token_mlp='leff'
    att_se=False
    modulator=False
    vit_dim=256
    vit_depth=12
    vit_nheads=8
    vit_mlp_dim=512
    vit_patch_size=16
    global_skip=False
    local_skip=False
    vit_share=False
    train_ps=128
    val_ps=128
    resume=False
    train_dir='./datasets/SIDD/train'
    val_dir='./datasets/SIDD/val'
    warmup=False
    warmup_epochs=3
    local_rank=-1
    distribute=True 
    distribute_mode='DDP'
    
        
        


In [ ]:
opt=Options()
opt.arch="Uformer_B"
opt.batch_size=16
opt.gpu="0"
opt.train_ps=128
opt.train_dir="/kaggle/working/datasets/denoising/sidd/train"
opt.env="_0706"
opt.val_dir="/kaggle/input/sidd-val/val"
opt.save_dir="/kaggle/working/logs/"
opt.dataset="sidd"
opt.warmup=True




In [ ]:
from glob import glob
from tqdm import tqdm
import numpy as np
import os
from natsort import natsorted
import cv2
from joblib import Parallel, delayed
import multiprocessing
import argparse
new_path = "/kaggle/input/uformer-main/Uformer-main"
os.chdir(new_path)
# 打印当前工作目录
import utils
from dataset.dataset_denoise import *
######### Set GPUs ###########
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"#opt.gpu  用双卡T4
import torch
# 是一个PyTorch设置，用于在训练神经网络时启用cuDNN的自动优化。当设置为True时，cuDNN会自动选择最佳的卷积算法和
# 内存分配策略，以提高计算性能。
torch.backends.cudnn.benchmark = True

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from natsort import natsorted
import glob
import random
import time
import numpy as np
from einops import rearrange, repeat
import datetime
from pdb import set_trace as stx

from losses import CharbonnierLoss

from tqdm import tqdm 
from warmup_scheduler import GradualWarmupScheduler
from torch.optim.lr_scheduler import StepLR
from timm.utils import NativeScaler
print("加载完了")
# from utils.loader import  get_training_data,get_validation_data

In [ ]:
log_dir = os.path.join("/kaggle/working/logs/", "denoising", "sidd", "Uformer_B_0706")
if not os.path.exists(log_dir):
    os.makedirs(log_dir)  #建立一个log_dir
    print("建立成功")
logname = os.path.join(log_dir, datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")+'.txt')
print("Now time is : ",datetime.datetime.now().isoformat())
result_dir = os.path.join(log_dir, 'results') #./logs/denoising/sidd/Uformer_B/_0706/results
model_dir = os.path.join(log_dir, 'models')#./logs/denoising/sidd/Uformer_B/_0706/models
utils.mkdir(result_dir)  #建立
utils.mkdir(model_dir)
print("建立成功")

In [ ]:
random.seed(1234)
np.random.seed(1234)
torch.manual_seed(1234)
torch.cuda.manual_seed_all(1234)

In [ ]:
model_restoration = utils.get_arch(opt)   #返回模型Uformer_B

with open(logname,'a') as f:
    f.write(str(opt)+'\n')
    f.write(str(model_restoration)+'\n')

In [ ]:
start_epoch = 1
if opt.optimizer.lower() == 'adam':
    optimizer = optim.Adam(model_restoration.parameters(), lr=opt.lr_initial, betas=(0.9, 0.999),eps=1e-8, weight_decay=opt.weight_decay)
elif opt.optimizer.lower() == 'adamw':
        optimizer = optim.AdamW(model_restoration.parameters(), lr=opt.lr_initial, betas=(0.9, 0.999),eps=1e-8, weight_decay=opt.weight_decay)
else:
    raise Exception("Error optimizer...")

In [ ]:
model_restoration = torch.nn.DataParallel (model_restoration) #单个gpu也可以用
model_restoration.cuda() 

In [ ]:
if opt.warmup:
    print("Using warmup and cosine strategy!")
    warmup_epochs = opt.warmup_epochs
    scheduler_cosine = optim.lr_scheduler.CosineAnnealingLR(optimizer, opt.nepoch-warmup_epochs, eta_min=1e-6)
    scheduler = GradualWarmupScheduler(optimizer, multiplier=1, total_epoch=warmup_epochs, after_scheduler=scheduler_cosine)
    scheduler.step()
else:
    step = 50
    print("Using StepLR,step={}!".format(step))
    '''
    StepLR调度器对象。StepLR 是一种学习率调度器，它在训练过程中按照给定的步长（step_size）和衰减因子（gamma）
    调整学习率。step_size: 指定多少个epoch更新一次学习率。gamma: 学习率每次更新时乘以的因子。
    '''
    scheduler = StepLR(optimizer, step_size=step, gamma=0.5)
    scheduler.step() #更新学习率

In [ ]:
if opt.resume:  #false
    path_chk_rest = opt.pretrain_weights 
    print("Resume from "+path_chk_rest)
    utils.load_checkpoint(model_restoration,path_chk_rest) 
    start_epoch = utils.load_start_epoch(path_chk_rest) + 1 
    lr = utils.load_optim(optimizer, path_chk_rest) 

    # for p in optimizer.param_groups: p['lr'] = lr 
    # warmup = False 
    # new_lr = lr 
    # print('------------------------------------------------------------------------------') 
    # print("==> Resuming Training with learning rate:",new_lr) 
    # print('------------------------------------------------------------------------------') 
    for i in range(1, start_epoch):
        scheduler.step()
    new_lr = scheduler.get_lr()[0]
    print('------------------------------------------------------------------------------')
    print("==> Resuming Training with learning rate:", new_lr)
    print('------------------------------------------------------------------------------')

    # scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, opt.nepoch-start_epoch+1, eta_min=1e-6) 

In [ ]:
criterion = CharbonnierLoss().cuda()

In [ ]:
print('===> Loading datasets')
img_options_train = {'patch_size':opt.train_ps}   #128
train_dataset = get_training_data(opt.train_dir, img_options_train)
train_loader = DataLoader(dataset=train_dataset, batch_size=opt.batch_size, shuffle=True, 
        num_workers=opt.train_workers, pin_memory=False, drop_last=False)
val_dataset = get_validation_data(opt.val_dir)
val_loader = DataLoader(dataset=val_dataset, batch_size=opt.batch_size, shuffle=False, 
        num_workers=opt.eval_workers, pin_memory=False, drop_last=False)

len_trainset = train_dataset.__len__()
len_valset = val_dataset.__len__()
print("Sizeof training set: ", len_trainset,", sizeof validation set: ", len_valset)

In [ ]:
with torch.no_grad():
    model_restoration.eval()
    psnr_dataset = []
    psnr_model_init = []
    for ii, data_val in enumerate((val_loader), 0):
        target = data_val[0].cuda()
        input_ = data_val[1].cuda()
        with torch.cuda.amp.autocast():
            restored = model_restoration(input_)
            restored = torch.clamp(restored,0,1)  
        psnr_dataset.append(utils.batch_PSNR(input_, target, False).item())
        psnr_model_init.append(utils.batch_PSNR(restored, target, False).item())
    psnr_dataset = sum(psnr_dataset)/len_valset
    psnr_model_init = sum(psnr_model_init)/len_valset
    print('Input & GT (PSNR) -->%.4f dB'%(psnr_dataset), ', Model_init & GT (PSNR) -->%.4f dB'%(psnr_model_init))

In [ ]:
print('===> Start Epoch {} End Epoch {}'.format(start_epoch,opt.nepoch))
best_psnr = 0
best_epoch = 0
best_iter = 0
eval_now = len(train_loader)//4
print("\nEvaluation after every {} Iterations !!!\n".format(eval_now))

loss_scaler = NativeScaler()
torch.cuda.empty_cache()
for epoch in range(start_epoch, opt.nepoch + 1):  #训练250轮
    epoch_start_time = time.time()
    epoch_loss = 0
    train_id = 1

    for i, data in enumerate(tqdm(train_loader), 0): 
        # zero_grad
        optimizer.zero_grad()

        target = data[0].cuda()
        input_ = data[1].cuda()

        if epoch>5:     #如果已经训练了5轮以上
            target, input_ = utils.MixUp_AUG().aug(target, input_)
        with torch.cuda.amp.autocast():
            restored = model_restoration(input_)
            loss = criterion(restored, target)
        loss_scaler(
                loss, optimizer,parameters=model_restoration.parameters())
        epoch_loss +=loss.item()

        #### Evaluation ####    过了val_now遍数据就评估一下模型
        if (i+1)%eval_now==0 and i>0:
            with torch.no_grad():
                model_restoration.eval()
                psnr_val_rgb = []
                for ii, data_val in enumerate((val_loader), 0):
                    target = data_val[0].cuda()
                    input_ = data_val[1].cuda()
                    filenames = data_val[2]
                    with torch.cuda.amp.autocast():
                        restored = model_restoration(input_)
                    restored = torch.clamp(restored,0,1)  
                    psnr_val_rgb.append(utils.batch_PSNR(restored, target, False).item())

                psnr_val_rgb = sum(psnr_val_rgb)/len_valset
                
                if psnr_val_rgb > best_psnr:
                    best_psnr = psnr_val_rgb
                    best_epoch = epoch
                    best_iter = i 
                    torch.save({'epoch': epoch, 
                                'state_dict': model_restoration.state_dict(),
                                'optimizer' : optimizer.state_dict()
                                }, os.path.join(model_dir,"model_best.pth"))

                print("[Ep %d it %d\t PSNR SIDD: %.4f\t] ----  [best_Ep_SIDD %d best_it_SIDD %d Best_PSNR_SIDD %.4f] " % (epoch, i, psnr_val_rgb,best_epoch,best_iter,best_psnr))
                with open(logname,'a') as f:
                    f.write("[Ep %d it %d\t PSNR SIDD: %.4f\t] ----  [best_Ep_SIDD %d best_it_SIDD %d Best_PSNR_SIDD %.4f] " \
                        % (epoch, i, psnr_val_rgb,best_epoch,best_iter,best_psnr)+'\n')
                model_restoration.train()
                torch.cuda.empty_cache()
    scheduler.step()  #更新学习率

    #每训练完一轮保存一下记录之类的信息
    print("------------------------------------------------------------------")
    print("Epoch: {}\tTime: {:.4f}\tLoss: {:.4f}\tLearningRate {:.6f}".format(epoch, time.time()-epoch_start_time,epoch_loss, scheduler.get_lr()[0]))
    print("------------------------------------------------------------------")
    with open(logname,'a') as f:
        f.write("Epoch: {}\tTime: {:.4f}\tLoss: {:.4f}\tLearningRate {:.6f}".format(epoch, time.time()-epoch_start_time,epoch_loss, scheduler.get_lr()[0])+'\n')

    torch.save({'epoch': epoch, 
                'state_dict': model_restoration.state_dict(),
                'optimizer' : optimizer.state_dict()
                }, os.path.join(model_dir,"model_latest.pth"))   

    if epoch%opt.checkpoint == 0:
        torch.save({'epoch': epoch, 
                    'state_dict': model_restoration.state_dict(),
                    'optimizer' : optimizer.state_dict()
                    }, os.path.join(model_dir,"model_epoch_{}.pth".format(epoch))) 
print("Now time is : ",datetime.datetime.now().isoformat())